In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Make a reference to the books.csv file path
csv_path = "Data/annual_conc_by_monitor_2019.csv"

# Import the books.csv file as a DataFrame
air_df = pd.read_csv(csv_path, encoding="utf-8")
air_df.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,1 HOUR,...,0.054,0.045,0.029,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
3,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
4,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,9.500,7.300,3.800,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",3/10/2020


In [3]:
# Remove unecessary columns from the DataFrame and save the new DataFrame
reduced_df = air_df[["State Code", "County Code", "Site Num", "Latitude",
                       "Longitude", "Parameter Name", "Sample Duration", 
                     "Metric Used", "Units of Measure", "Arithmetic Mean", 
                     "Arithmetic Standard Dev", "POC", "State Name", "County Name", "City Name"]]
reduced_df.head()

,State Code,County Code,Site Num,Latitude,Longitude,Parameter Name,Sample Duration,Metric Used,Units of Measure,Arithmetic Mean,Arithmetic Standard Dev,POC,State Name,County Name,City Name
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,0.012048,1,Alabama,Baldwin,Fairhope
1,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8 hour running average of obs...,Parts per million,0.040390,0.011461,1,Alabama,Baldwin,Fairhope
2,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8 hour running average of obs...,Parts per million,0.040390,0.011461,1,Alabama,Baldwin,Fairhope
3,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8-hour running average,Parts per million,0.040489,0.011407,1,Alabama,Baldwin,Fairhope
4,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope


In [4]:
# Chamge column names with lower cases for later work
reduced_df = reduced_df.rename(columns = {"State Code": "state_code", "County Code": "county_code", "Site Num": "site_num", 
                                         "Latitude": "latitude", "Longitude": "longitude", "Parameter Name": "parameter_name",
                                         "Sample Duration": "sample_duration", "Metric Used": "metric_used", "Units of Measure": "units",
                                         "Arithmetic Mean": "mean", "Arithmetic Standard Dev": "stv",
                                         "POC": "poc", "State Name": "state_name", "County Name": "county_name", "City Name": "city_name"})
reduced_df.head()

,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,0.012048,1,Alabama,Baldwin,Fairhope
1,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8 hour running average of obs...,Parts per million,0.040390,0.011461,1,Alabama,Baldwin,Fairhope
2,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8 hour running average of obs...,Parts per million,0.040390,0.011461,1,Alabama,Baldwin,Fairhope
3,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8-hour running average,Parts per million,0.040489,0.011407,1,Alabama,Baldwin,Fairhope
4,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope


In [5]:
# Clean up ozone dataframe
ozone_df = reduced_df.loc[reduced_df['parameter_name'] == 'Ozone']
ozone_df.head()

,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,0.012048,1,Alabama,Baldwin,Fairhope
1,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8 hour running average of obs...,Parts per million,0.040390,0.011461,1,Alabama,Baldwin,Fairhope
2,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8 hour running average of obs...,Parts per million,0.040390,0.011461,1,Alabama,Baldwin,Fairhope
3,1,3,10,30.497478,-87.880258,Ozone,8-HR RUN AVG BEGIN HOUR,Daily maximum of 8-hour running average,Parts per million,0.040489,0.011407,1,Alabama,Baldwin,Fairhope
12,1,33,1002,34.762619,-87.638097,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.043192,0.009939,1,Alabama,Colbert,Muscle Shoals


In [6]:
ozone_final_df = ozone_df.loc[ozone_df['sample_duration'] == '1 HOUR']
ozone_final_df.head(20)

,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,0.012048,1,Alabama,Baldwin,Fairhope
12,1,33,1002,34.762619,-87.638097,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.043192,0.009939,1,Alabama,Colbert,Muscle Shoals
24,1,49,9991,34.289001,-85.970065,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048427,0.010055,1,Alabama,DeKalb,Crossville
28,1,51,4,32.535680,-86.255193,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.045878,0.011226,1,Alabama,Elmore,Wetumpka
43,1,55,11,33.904039,-86.053867,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048975,0.011661,1,Alabama,Etowah,Southside
51,1,69,4,31.188933,-85.423094,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.040938,0.011056,1,Alabama,Houston,Dothan
77,1,73,23,33.553056,-86.815000,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.045349,0.013174,1,Alabama,Jefferson,Birmingham
245,1,73,1003,33.485556,-86.915000,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.050222,0.014085,1,Alabama,Jefferson,Fairfield
249,1,73,1005,33.331111,-87.003611,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.049355,0.013261,1,Alabama,Jefferson,NaN
269,1,73,1010,33.545278,-86.549167,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048765,0.011909,1,Alabama,Jefferson,Leeds


In [7]:
# Clean up pm2.5 dataframe
pm_df = reduced_df.loc[reduced_df['parameter_name'] == 'PM2.5 - Local Conditions']
pm_df.head()

,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
4,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
5,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
6,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Quarterly Means of Daily Means,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
7,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Quarterly Means of Daily Means,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
8,1,27,1,33.284928,-85.803608,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550467,3.442393,1,Alabama,Clay,Ashland


In [8]:
pm_2_df = pm_df.loc[pm_df['metric_used'] == 'Daily Mean']
pm_2_df.head(20)

,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
4,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
5,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
8,1,27,1,33.284928,-85.803608,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550467,3.442393,1,Alabama,Clay,Ashland
9,1,27,1,33.284928,-85.803608,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550467,3.442393,1,Alabama,Clay,Ashland
16,1,33,1002,34.762619,-87.638097,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1,Alabama,Colbert,Muscle Shoals
17,1,33,1002,34.762619,-87.638097,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1,Alabama,Colbert,Muscle Shoals
20,1,49,1003,34.288567,-85.969858,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550000,3.508534,1,Alabama,DeKalb,Crossville
21,1,49,1003,34.288567,-85.969858,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550000,3.508534,1,Alabama,DeKalb,Crossville
39,1,55,10,33.991494,-85.992647,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.410526,3.999012,1,Alabama,Etowah,Gadsden
40,1,55,10,33.991494,-85.992647,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.410526,3.999012,1,Alabama,Etowah,Gadsden


In [9]:
# Drop duplicates data for each location
pm_3_df = pm_2_df.drop_duplicates()
pm_3_df.head(20)


,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
4,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
8,1,27,1,33.284928,-85.803608,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550467,3.442393,1,Alabama,Clay,Ashland
16,1,33,1002,34.762619,-87.638097,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1,Alabama,Colbert,Muscle Shoals
20,1,49,1003,34.288567,-85.969858,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550000,3.508534,1,Alabama,DeKalb,Crossville
39,1,55,10,33.991494,-85.992647,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.410526,3.999012,1,Alabama,Etowah,Gadsden
47,1,69,3,31.224783,-85.390789,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.115652,3.356993,1,Alabama,Houston,Dothan
115,1,73,23,33.553056,-86.815000,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),10.310833,4.726695,1,Alabama,Jefferson,Birmingham
119,1,73,23,33.553056,-86.815000,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),10.100000,4.923723,2,Alabama,Jefferson,Birmingham
124,1,73,23,33.553056,-86.815000,PM2.5 - Local Conditions,24-HR BLK AVG,Daily Mean,Micrograms/cubic meter (LC),10.552222,4.526788,3,Alabama,Jefferson,Birmingham
259,1,73,1005,33.331111,-87.003611,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.701667,3.471492,1,Alabama,Jefferson,NaN


In [10]:
# There are still some duplicate data, so used POC column and removed the duplicates
pm_final_df = pm_3_df.loc[pm_3_df['poc'] == 1]
pm_final_df.head(20)

,state_code,county_code,site_num,latitude,longitude,parameter_name,sample_duration,metric_used,units,mean,stv,poc,state_name,county_name,city_name
4,1,3,10,30.497478,-87.880258,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
8,1,27,1,33.284928,-85.803608,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550467,3.442393,1,Alabama,Clay,Ashland
16,1,33,1002,34.762619,-87.638097,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1,Alabama,Colbert,Muscle Shoals
20,1,49,1003,34.288567,-85.969858,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.550000,3.508534,1,Alabama,DeKalb,Crossville
39,1,55,10,33.991494,-85.992647,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.410526,3.999012,1,Alabama,Etowah,Gadsden
47,1,69,3,31.224783,-85.390789,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.115652,3.356993,1,Alabama,Houston,Dothan
115,1,73,23,33.553056,-86.815000,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),10.310833,4.726695,1,Alabama,Jefferson,Birmingham
259,1,73,1005,33.331111,-87.003611,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.701667,3.471492,1,Alabama,Jefferson,NaN
280,1,73,1010,33.545278,-86.549167,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.363934,3.695269,1,Alabama,Jefferson,Leeds
320,1,73,2003,33.499722,-86.924167,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.705000,3.555251,1,Alabama,Jefferson,Birmingham


In [11]:
# There are still some duplicates (ex. id = 845 and 846).
# But at this point, I decided to move forward to merge ozone and pm dataframes with 'outer' way.
merge_df = pd.merge(ozone_final_df, pm_final_df, on=["state_code", "county_code", "site_num", "latitude", "longitude"], how="outer")
merge_df

,state_code,county_code,site_num,latitude,longitude,parameter_name_x,sample_duration_x,metric_used_x,units_x,mean_x,...,parameter_name_y,sample_duration_y,metric_used_y,units_y,mean_y,stv_y,poc_y,state_name_y,county_name_y,city_name_y
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1.0,Alabama,Baldwin,Fairhope
1,1,33,1002,34.762619,-87.638097,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.043192,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1.0,Alabama,Colbert,Muscle Shoals
2,1,49,9991,34.289001,-85.970065,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,51,4,32.535680,-86.255193,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.045878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,55,11,33.904039,-86.053867,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,72,53,3,18.383983,-65.618888,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),6.868067,4.565306,1.0,Puerto Rico,Fajardo,NaN
1657,72,57,12,17.955378,-66.162122,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),5.303571,3.838931,1.0,Puerto Rico,Guayama,Guayama
1658,72,61,5,18.440095,-66.114460,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.487179,3.999343,1.0,Puerto Rico,Guaynabo,Guaynabo
1659,72,113,4,18.009558,-66.627249,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),6.285841,3.459430,1.0,Puerto Rico,Ponce,Ponce


In [12]:
# Merge ozone and pm dataframes with left way
merge_2_df = pd.merge(ozone_final_df, pm_final_df, on=["state_code", "county_code", "site_num", "latitude", "longitude"], how="left")
merge_2_df

,state_code,county_code,site_num,latitude,longitude,parameter_name_x,sample_duration_x,metric_used_x,units_x,mean_x,...,parameter_name_y,sample_duration_y,metric_used_y,units_y,mean_y,stv_y,poc_y,state_name_y,county_name_y,city_name_y
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1.0,Alabama,Baldwin,Fairhope
1,1,33,1002,34.762619,-87.638097,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.043192,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1.0,Alabama,Colbert,Muscle Shoals
2,1,49,9991,34.289001,-85.970065,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,51,4,32.535680,-86.255193,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.045878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,55,11,33.904039,-86.053867,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,56,45,3,43.873056,-104.191944,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1311,72,21,10,18.420089,-66.150615,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.020147,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),6.820513,4.155834,1.0,Puerto Rico,Bayamon,Bayamon
1312,72,33,8,18.440774,-66.126531,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.012317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1313,72,97,7,18.216038,-67.144107,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.010220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Merge ozone and pm dataframes with right way
merge_3_df = pd.merge(ozone_final_df, pm_final_df, on=["state_code", "county_code", "site_num", "latitude", "longitude"], how="right")
merge_3_df

,state_code,county_code,site_num,latitude,longitude,parameter_name_x,sample_duration_x,metric_used_x,units_x,mean_x,...,parameter_name_y,sample_duration_y,metric_used_y,units_y,mean_y,stv_y,poc_y,state_name_y,county_name_y,city_name_y
0,1,3,10,30.497478,-87.880258,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.044808,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.551402,3.148146,1,Alabama,Baldwin,Fairhope
1,1,33,1002,34.762619,-87.638097,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.043192,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.213235,3.127724,1,Alabama,Colbert,Muscle Shoals
2,1,73,23,33.553056,-86.815000,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.045349,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),10.310833,4.726695,1,Alabama,Jefferson,Birmingham
3,1,73,1005,33.331111,-87.003611,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.049355,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.701667,3.471492,1,Alabama,Jefferson,NaN
4,1,73,1010,33.545278,-86.549167,Ozone,1 HOUR,Daily maxima of observed hourly values (betwee...,Parts per million,0.048765,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),8.363934,3.695269,1,Alabama,Jefferson,Leeds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,72,53,3,18.383983,-65.618888,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),6.868067,4.565306,1,Puerto Rico,Fajardo,NaN
673,72,57,12,17.955378,-66.162122,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),5.303571,3.838931,1,Puerto Rico,Guayama,Guayama
674,72,61,5,18.440095,-66.114460,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),7.487179,3.999343,1,Puerto Rico,Guaynabo,Guaynabo
675,72,113,4,18.009558,-66.627249,NaN,NaN,NaN,NaN,NaN,...,PM2.5 - Local Conditions,24 HOUR,Daily Mean,Micrograms/cubic meter (LC),6.285841,3.459430,1,Puerto Rico,Ponce,Ponce


In [14]:
# I used merge df with outer join. This method included more rows. I make csv file to make graphs now
merge_df.to_csv('merge.csv', index=False, header=True)

In [15]:
from sqlalchemy import create_engine

In [20]:
# Connect to the database
engine = create_engine('sqlite:///aqi.sqlite', echo=False)
# I also created a db file:
# engine = create_engine('sqlite:///aqi.db', echo=False)
sqlite_connection = engine.connect()

In [21]:
merge_df.to_sql('aqi', con=engine)

In [22]:
engine.execute("SELECT * FROM aqi").fetchall()

[(0, 1, 3, 10, 30.497478000000005, -87.880258, 'Ozone', '1 HOUR', 'Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM)', 'Parts per million', 0.044808, 0.012048, 1.0, 'Alabama', 'Baldwin', 'Fairhope', 'PM2.5 - Local Conditions', '24 HOUR', 'Daily Mean', 'Micrograms/cubic meter (LC)', 7.551402, 3.148146, 1.0, 'Alabama', 'Baldwin', 'Fairhope'),
 (1, 1, 33, 1002, 34.762619, -87.638097, 'Ozone', '1 HOUR', 'Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM)', 'Parts per million', 0.043192, 0.009939, 1.0, 'Alabama', 'Colbert', 'Muscle Shoals', 'PM2.5 - Local Conditions', '24 HOUR', 'Daily Mean', 'Micrograms/cubic meter (LC)', 7.213235000000001, 3.127724, 1.0, 'Alabama', 'Colbert', 'Muscle Shoals'),
 (2, 1, 49, 9991, 34.289001, -85.970065, 'Ozone', '1 HOUR', 'Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM)', 'Parts per million', 0.048427, 0.010055, 1.0, 'Alabama', 'DeKalb', 'Crossville', None, None, None, None, None, None, None, None, No

In [23]:
engine.table_names()

['aqi']